In [3]:
from TP1 import scrape_financial_ratios,get_average_return,download_all_historical_data
import json
from datetime import date
import pandas as pd
from TP3 import apply_labeling_to_folder,preprocess_data,train_and_evaluate_models,train_random_forest_model,predict_today
from TP2 import preprocess_financial_clustering,do_kmeans_clustering,do_hierarchical_clustering,prepare_returns_data,preprocess_risk_clustering
from TP4 import predict as predict4
from TP5 import predict as predict5

In [4]:
folder_path = "data/Companies_Historical_Data/"




In [5]:
R=open("report.html", "w")
R.write(f"<h1>Report du {date.today()}</h1>")

with open('data/companies.json', 'r') as file:
    data = json.load(file)
ratio_keys = [
    "forwardPE", "beta", "priceToBook", "priceToSales", "dividendYield",
    "trailingEps", "debtToEquity", "currentRatio", "quickRatio",
    "returnOnEquity", "returnOnAssets", "operatingMargins", "profitMargins"
]
ratios=scrape_financial_ratios(data,ratio_keys)
file_path = "data/ratios_financiers.csv"
returns_folder = "data/Companies_Historical_Data/"
data_financial, names_financial = preprocess_financial_clustering(file_path)
fincluster=do_kmeans_clustering(data_financial, names_financial, n_clusters=5,show_graph=False)

labeled_data = apply_labeling_to_folder(folder_path )
print("Prétraitement et extraction des caractéristiques...")
X, y = preprocess_data(labeled_data)
model, scaler= train_random_forest_model(X, y)


Scraping des ratios financiers...
data/Companies_Historical_Data/
Prétraitement et extraction des caractéristiques...
Processing stock: Adobe, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: Alibaba, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: Alphabet, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: Amazon, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: AMD, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: Apple, initial shape: (1235, 5)
After adding indicators, shape: (1235, 14)
After dropping NaN, shape: (1202, 14)
Processing stock: ASML, initial shape: (1261, 5)
After adding indicators, shape: (1261, 14)


In [ ]:

for company, row in ratios.iterrows():
    
    R.write(f"<h2>Entreprise : {company}</h2>")
    R.write("<h3>Ratios Importants:</h3><ul>")

    for ratio in ratios.columns:
        value = row[ratio]
        if pd.notnull(value): 
            R.write(f"<li>{ratio} : {value}</li>")

    R.write("</ul>")
    print(data[company])
    R.write(f"<h3>Rendement journalier moyen: {get_average_return({data[company]})}</h3>")
    
    if company in fincluster["Name"].values:
        cluster_id = fincluster.loc[fincluster["Name"] == company, "Cluster"].values[0]
        same_cluster_companies = fincluster[fincluster["Cluster"] == cluster_id]["Name"].tolist()
        same_cluster_companies = [name for name in same_cluster_companies if name != company]
        R.write(f"<h3>Entreprise semblables (profils financiers) :</h3><p>{", ".join(same_cluster_companies)}</p> ")
    else :
        R.write("<h3>Entreprise semblables (profils financiers) :</h3><p>Aucune</p> ")
    _,action=predict_today(f"data\Companies_Historical_Data\{company}.csv")
    R.write("<h3>conseil d’investissement :</h3>")
    if action=="HOLD":
        R.write("<h4 style='color: yellow'> Hold </h4>")
    elif action=="SELL":
        R.write("<h4 style='color: red'> Vente </h4>")
    else:
        R.write("<h4 style='color: green'> Achat </h4>")

    prediction4=predict4(company)
    print(prediction4)
    prediction5=float(predict5(company))
    R.write(f"<h3>Prediciton du prix de demain :</h3><p>{(prediction5+float(prediction4['XGBoost'])+float(prediction4['RandomForest'])+float(prediction4['KNN']))/4}</p> ")


[*********************100%***********************]  1 of 1 completed

AAPL


Trading signal for today (1254): HOLD
{'XGBoost': np.float32(186.94073), 'RandomForest': np.float64(193.76325073242182), 'KNN': np.float64(186.21830139160159)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
MSFT


[*********************100%***********************]  1 of 1 completed


Trading signal for today (1254): HOLD
{'XGBoost': np.float32(414.32248), 'RandomForest': np.float64(416.71330566406243), 'KNN': np.float64(407.31427001953125)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
AMZN


...

Trading signal for today (1254): HOLD
{'XGBoost': np.float32(148.41719), 'RandomForest': np.float64(148.49034881591797), 'KNN': np.float64(149.09443054199218)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
PFE


[*********************100%***********************]  1 of 1 completed


Trading signal for today (1254): BUY
{'XGBoost': np.float32(24.889647), 'RandomForest': np.float64(24.36674419403076), 'KNN': np.float64(25.96884346008301)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
XOM


[*********************100%***********************]  1 of 1 completed


Trading signal for today (1254): HOLD
